In [38]:
# Imports
import sys
import gurobipy as gp
import numpy as np

In [39]:
# Parameters (Valus of Instance 1)
NUM_VAGOES = 2
NUM_TRECHOS = 6
NUM_ATIVIDADES = 15
NUM_TRENS = 2
P = 5 # tempo de circulação de um vagão j em um trecho q
M = 2880
VAGOES_TREM_SAIDA = [2,2]
VAGOES_TREM_CHEGADA = [1,1]

In [41]:
# Generate AMV
# AMV[q,s] = 1 se o trecho q é ligado ao trecho s (q vizinho de s ou existe AMV entre q e s); 0 caso contrário
AMV = [[1,1,0,0,0,0],[1,1,1,0,1,0],[0,1,1,1,0,0],[0,0,1,1,0,0],[0,1,0,0,1,1],[0,0,0,0,1,1]]

In [42]:
# Generate PSA
# PSA[j] = trecho do pátio onde o vagão j deverá estar posicionado para que o trem t ∈ Tβ |j ∈ t possa sair
PSA = [1,2]

In [43]:
# Generate PCH
# PCH[j] = trecho do pátio onde o vagão j estará posicionado ao receber o trem t ∈ Tα|j ∈ t
PCH = [2,1]

In [44]:
# Generate Release
# RELEASE[j] = instante mais cedo que o vagão j está disponível para entrar no pátio (release date), ou seja, horário de chegada do trem t ∈ Tα|j ∈ t
RELEASE = [10,10]

In [46]:
# Auxiliary constants
FIRST_OP = 1
LAST_OP = NUM_ATIVIDADES - 1

In [47]:
# Simplify instances informed values
n = NUM_VAGOES
m = NUM_TRECHOS
o = NUM_ATIVIDADES
v = NUM_TRENS
ns = VAGOES_TREM_SAIDA
nc = VAGOES_TREM_CHEGADA

In [48]:
# Initialize model
model = gp.Model("milp")

In [49]:
# Declare decision variables
var_y = model.addVar(vtype=gp.GRB.CONTINUOUS, name="y")          ## y[i,j] = instante de início da i-ésima operação do vagão j
var_z = model.addVar(vtype=gp.GRB.BINARY, name="z")              ## z[i,q,j] = 1 se a i-ésima operação do vagão j acontece no trecho q; 0 caso contrário
var_x = model.addVar(vtype=gp.GRB.BINARY, name="x")              ## x[q,i,j,k,l] = 1 se a i-ésima operação do vagão j precede a k-ésima operação do vagão l no trecho q; 0 caso contrário
var_f = model.addVar(vtype=gp.GRB.BINARY, name="f")              ## f[i,j] = 1 se a i-ésima operação do vagão j é a última, antes da partida do trem t ∈ Tβ |j ∈ t; 0 caso contrário
var_c = model.addVar(vtype=gp.GRB.CONTINUOUS, name="c")          ## C[j] = instante de conclusão do vagão j, horário de partida do trem t ∈ Tβ|j ∈ t
var_c_max = model.addVar(vtype=gp.GRB.CONTINUOUS, name="c_max")  ## Cmax = horário da partida do último vagão j ∈ t, t ∈ Tβ (makespan)

In [50]:
# Define objective function
model.setObjective(var_c_max, gp.GRB.MINIMIZE)

In [72]:
# C1
c1 = model.addConstr
(
  (
    var_y[i+1,j] >= var_y[i,j] 
    + ( P * gp.quicksum(var_x[i,q,j] for q in range(m)) )
  )
  for j in range(n) 
  for i in range(o)
  # if i < o
)

<generator object <genexpr> at 0x7fe608fb5f20>

In [52]:
# C2
c2 = model.addConstr
(
  (
    var_y[i+1,j] 
    >= var_y[k,l]
    - (M * var_x[q,i,j,k,l])
    - (M * (1 - var_z[i,q,j]))
    - (M * (1 - var_z[k,q,l]))
  )
  for j,l in range(n) 
  for k,i in range(o)
  for q in range(m)
  if i < o
)

<generator object <genexpr> at 0x7fe608fabdd0>

In [53]:
# C3
c3 = model.addConstr
(
  (
    var_y[k,l] 
    >= var_y[i+1,j]
    - (M * (1 - var_x[q,i,j,k,l]))
    - (M * (1 - var_z[i,q,j]))
    - (M * (1 - var_z[k,q,l]))
  )
  for j,l in range(n) 
  for k,i in range(o)
  for q in range(m)
  if i < o
)

<generator object <genexpr> at 0x7fe608fabf90>

In [54]:
# C4
c4 = model.addConstr
(
  (
    gp.quicksum(var_z[FIRST_OP,q,j] for q in range(m))
    <= 1
  )
  for j in range(n) 
  # for i in range(o)
  # if i == 1
)

<generator object <genexpr> at 0x7fe5f9f560b0>

In [55]:
# C5
c5 = model.addConstr
(
  (
    var_x[q,i,j,k,l] 
    + gp.quicksum(var_x[s,k-1,l,i+1,j] for s in range(m))
    <= 1
  )
  for j,l in range(n) 
  for k,i in range(o)
  for q in range(m)
  if i < o and k > FIRST_OP and j != l
)

<generator object <genexpr> at 0x7fe5f9f56270>

In [56]:
# C6
c6 = model.addConstr
(
  (
    var_x[q,i,j,k,l] 
    + var_x[q,k,l,i,j] 
    >= 1
    - (M * (1 - var_z[i,q,j]))
    - (M * (1 - var_z[k,q,l]))
  )
  for j,l in range(n) 
  for k,i in range(o)
  for q in range(m)
  if j != l
)

<generator object <genexpr> at 0x7fe608fb5120>

In [57]:
# C7
c7 = model.addConstr
(
  (
    gp.quicksum(var_z[i,q,j] for q in range(m))
    == 1
    - gp.quicksum(var_f[k,j] for k in range(i-1))
  )
  for j in range(n) 
  for i in range(o)
  if i > FIRST_OP
)

<generator object <genexpr> at 0x7fe5f9f562e0>

In [58]:
# C8
c8 = model.addConstr
(
  (
   var_f[i,j]
   <= var_z[i,PSA[j],j]
  )
  for j in ns
  for i in range(o)
  # for q in range(m)
  # if q == PSA[j]
)

<generator object <genexpr> at 0x7fe608fb5430>

In [59]:
# C9
c9 = model.addConstr
(
  (
   gp.quicksum(var_f[i,j] for i in range(o))
   == 1
  )
  for j in ns
)

<generator object <genexpr> at 0x7fe608fb5510>

In [60]:
# C10
c10 = model.addConstr
(
  (
   gp.quicksum(var_f[i,j] for i in range(o))
   == 0
  )
  for j in nc
)

<generator object <genexpr> at 0x7fe608fb5190>

In [61]:
# C11
c11 = model.addConstr
(
  (
   var_z[FIRST_OP,PCH[j],j]
   == 1
  )
  for j in range(n) 
  # for i in range(o) 
  # for q in range(m) 
  # if q == PCH[j] and i == 1
)

<generator object <genexpr> at 0x7fe608fb54a0>

In [62]:
# C12
c12 = model.addConstr
(
  (
   var_y[FIRST_OP,j]
   == RELEASE[j]
  )
  for j in ns
  # for i in range(o) 
  # if i == 1
)

<generator object <genexpr> at 0x7fe608fb57b0>

In [63]:
# C13
c13 = model.addConstr
(
  (
   var_y[FIRST_OP,j]
   >= RELEASE[j]
  )
  for j in ns 
  # for i in range(o) 
  # if i == 1
)

<generator object <genexpr> at 0x7fe608fb5890>

In [64]:
# C14
c14 = model.addConstr
(
  (
   var_y[FIRST_OP,j]
   == var_y[FIRST_OP,l]
  )
  for j,l in nc
  # for i in range(o)
  # if i == 1
)

<generator object <genexpr> at 0x7fe608fb5970>

In [65]:
# C15
c15 = model.addConstr
(
  (
   var_z[i+1,q,j]
   <= gp.quicksum(AMV[s,q] * var_z[i,s,j] for s in range(m))
  )
  for j in range(n) 
  for i in range(o)
  for q in range(m)
  # if i < o
)

<generator object <genexpr> at 0x7fe5f9f56ac0>

In [66]:
# C16
c16 = model.addConstr
(
  (
   var_c[j]
   >= var_y[i,l]
  )
  for j,l in ns
  for i in range(o)
  # if i < o
)

<generator object <genexpr> at 0x7fe608fb5a50>

In [67]:
# C17
c17 = model.addConstr
(
  (
   var_c[j]
   >= var_y[LAST_OP,j]
  )
  for j in nc 
  # for i in range(o)
  # if i == o
)

<generator object <genexpr> at 0x7fe608fb5c10>

In [68]:
# C18
c18 = model.addConstr
(
  (
   var_y[i+1,j]
   >= var_c[j] 
   - M 
   * (1 - gp.quicksum(var_f[k,j] for k in range(i)))
  )
  for j in range(n) 
  for i in range(o)
  # if i < o
)

<generator object <genexpr> at 0x7fe5f9f56f90>

In [69]:
# C19
c19 = model.addConstr
(
  (
   var_c_max >= var_c[j]
  )
  for j in ns
)

<generator object <genexpr> at 0x7fe608fb5b30>

In [70]:
# Execute model
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 6 columns and 0 nonzeros
Model fingerprint: 0x8293aa32
Variable types: 3 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
